In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from pandas.plotting import scatter_matrix
import seaborn as sns
import numpy as np
import pymongo

# Importation des données

In [ ]:
client = pymongo.MongoClient("mongodb://localhost:27017")
db = client["Tweet"]
user_collection = db["users"]

Suppression des colonnes non utilisées lors du kmeans

In [ ]:
users = list(user_collection.find({}))
users = pd.DataFrame(users)
users.index = users.user_id
users = users.drop(columns=["_id","user_id","last_tweet_published_id","tweet_ids","friends_count","followers_count"])
users.columns

In [ ]:
users.head()

On retire tweet_frequency de nos données sachant qu'il est corrélé avec l'agressivité. Ce paramètre n'apporte pas d'informations supplémentaires. Une visualisation de cette corrélation peut être observée dans le script ``previewData``

In [ ]:
users = users.drop(columns=["tweet_frequency"])

# Clustering avec KMeans

In [ ]:
model = KMeans(n_clusters=2)
model.fit(users)
inertia_scores = model.inertia_
print("Inertie:",inertia_scores)

In [ ]:
users["label"] = model.labels_
users["label"].value_counts()

In [ ]:
silhouette_score(users,model.labels_)

In [ ]:
users.groupby(model.labels_).mean()

# Exemple pour le moment, à modifier avec le cours ACP

# Mise en place de l'ACP pour representer les resultats du clustering

In [ ]:
acp = PCA(n_components=2)

In [ ]:
result =acp.fit_transform(X_CR)

In [ ]:
resultat = pd.DataFrame(result, columns = ["CP1","CP2"])
resultat.head()

# Presentation des clusters

In [ ]:
# Create scatter plot of `PC2` vs `PC1`
fig = px.scatter(
    data_frame = resultat,
    x= "CP1",
    y = "CP2",
    color = labels,
    title = "PCA Representation of Clusters"
)
fig.update_layout(xaxis_title = "Cluster", yaxis_title ="Value" )
fig.show()